### Classifier

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words_geo = stopwords.words('english')

/Users/tusharahuja/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [3]:
for i in ['as','at','by','between','to','from','in', 'off', 'there','where']:
    stop_words_geo.remove(i)

In [4]:
test_df_1 = pd.read_csv('./labelled_500.csv', usecols=['Query','geo'], nrows=499)
test_df_2 = pd.read_csv('./labelled_1000_correct.csv', usecols=['Query','geo'], nrows=1500)
test_df = test_df_1.append(test_df_2[499:1500])

In [5]:
df_nongeo_sampled = test_df[test_df['geo'] == 0].sample(n=319, frac=None, replace=False, weights=None, random_state=None, axis=None)

In [6]:
test_df_sampled = df_nongeo_sampled.append(test_df[test_df.geo == 1])

In [7]:
xtrain, xvalid, ytrain, yvalid = train_test_split(test_df_sampled.Query.values, test_df_sampled.geo.values, 
                                                  stratify=test_df_sampled.geo.values, 
                                                  random_state=42, 
                                                  test_size=0.2)

In [8]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = stop_words_geo)

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [10]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
print metrics.accuracy_score(yvalid,np.argmax(predictions, axis=1))

0.765625


In [12]:
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.83
Average recall score: 0.67


In [13]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = stop_words_geo)

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [14]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
print metrics.accuracy_score(yvalid,np.argmax(predictions, axis=1))

0.7265625


In [15]:
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.80
Average recall score: 0.61


In [16]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
print metrics.accuracy_score(yvalid,np.argmax(predictions, axis=1))

0.7421875


In [17]:
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.81
Average recall score: 0.62


In [18]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
print metrics.accuracy_score(yvalid,np.argmax(predictions, axis=1))

0.71875


In [19]:
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.79
Average recall score: 0.73


In [20]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [21]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)
print metrics.accuracy_score(yvalid,np.argmax(predictions, axis=1))

0.734375


In [22]:
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.80
Average recall score: 0.80


In [23]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print metrics.accuracy_score(yvalid,np.argmax(predictions, axis=1))

0.7265625


In [24]:
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.80
Average recall score: 0.75


In [25]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)
print metrics.accuracy_score(yvalid,np.argmax(predictions, axis=1))

0.75


In [26]:
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.81
Average recall score: 0.75


In [27]:
mll_scorer = metrics.make_scorer(np.mean(yvalid == np.argmax(predictions, axis=1)), greater_is_better=False, needs_proba=True)

In [28]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [02:52, 12740.97it/s]

Found 2196016 word vectors.


In [29]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower().decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words_geo]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [30]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]

100%|██████████| 128/128 [00:00<00:00, 2876.21it/s]


In [31]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [32]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)
np.mean(yvalid == np.argmax(predictions, axis=1))

0.8515625

In [33]:
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.89
Average recall score: 0.83


In [34]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)
np.mean(yvalid == np.argmax(predictions, axis=1))

0.8515625

In [35]:
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.89
Average recall score: 0.84


In [614]:
our_test = np.array(['cafe near google mountain view'])
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xtest_glove = [sent2vec(x) for x in tqdm(our_test)]

100%|██████████| 1/1 [00:00<00:00, 758.74it/s]


In [615]:
predictions = clf.predict_proba(xtest_glove)

In [616]:
np.argmax(predictions, axis=1)

array([0])

we see that a simple tuning of parameters can improve xgboost score on GloVe features! Believe me you can squeeze a lot more from it.

## Deep Learning

But this is an era of deep learning! We cant live without training a few neural networks. Here, we will train LSTM and a simple dense network on the GloVe features. Let's start with the dense network first:

In [37]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [38]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [39]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='softmax'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(2))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [40]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 510 samples, validate on 128 samples
Epoch 1/5
510/510 [==============================] - 1s 1ms/step - loss: 0.6128 - val_loss: 0.6850
Epoch 2/5
510/510 [==============================] - 0s 309us/step - loss: 0.4046 - val_loss: 0.6767
Epoch 3/5
510/510 [==============================] - 0s 263us/step - loss: 0.3132 - val_loss: 0.6691
Epoch 4/5
510/510 [==============================] - 0s 259us/step - loss: 0.2643 - val_loss: 0.6644


In [44]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [45]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 1507/1507 [00:00<00:00, 136501.02it/s]


In [47]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Fit the model with early stopping callback
earlystop = EarlyStopping(min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 510 samples, validate on 128 samples
Epoch 1/100
510/510 [==============================] - 8s 15ms/step - loss: 0.7093 - acc: 0.5157 - val_loss: 0.6624 - val_acc: 0.5234
Epoch 2/100
510/510 [==============================] - 7s 14ms/step - loss: 0.6921 - acc: 0.5412 - val_loss: 0.6394 - val_acc: 0.5312
Epoch 3/100
510/510 [==============================] - 7s 13ms/step - loss: 0.6755 - acc: 0.5510 - val_loss: 0.6126 - val_acc: 0.6484
Epoch 4/100
510/510 [==============================] - 5s 11ms/step - loss: 0.6525 - acc: 0.6137 - val_loss: 0.5827 - val_acc: 0.7266
Epoch 5/100
510/510 [==============================] - 5s 10ms/step - loss: 0.6222 - acc: 0.6627 - val_loss: 0.5484 - val_acc: 0.7656
Epoch 6/100
510/510 [==============================] - 5s 10ms/step - loss: 0.5911 - acc: 0.6941 - val_loss: 0.5124 - val_acc: 0.7812
Epoch 7/100
510/510 [==============================] - 5s 11ms/step - loss: 0.5632 - acc: 0.7373 - val_loss: 0.4793 - val_acc: 0.8047
Epoch 8/100
510/

### Accuracy is 89 percent

In [48]:
predictions = model.predict(xvalid_pad)
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.91
Average recall score: 0.95


In [49]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 510 samples, validate on 128 samples
Epoch 1/100
510/510 [==============================] - 11s 22ms/step - loss: 0.7135 - acc: 0.5137 - val_loss: 0.6728 - val_acc: 0.5938
Epoch 2/100
510/510 [==============================] - 10s 20ms/step - loss: 0.6845 - acc: 0.5412 - val_loss: 0.6471 - val_acc: 0.6016
Epoch 3/100
510/510 [==============================] - 10s 20ms/step - loss: 0.6619 - acc: 0.6098 - val_loss: 0.6203 - val_acc: 0.6406
Epoch 4/100
510/510 [==============================] - 10s 20ms/step - loss: 0.6520 - acc: 0.6118 - val_loss: 0.5897 - val_acc: 0.6875
Epoch 5/100
510/510 [==============================] - 10s 20ms/step - loss: 0.6154 - acc: 0.6647 - val_loss: 0.5538 - val_acc: 0.7734
Epoch 6/100
510/510 [==============================] - 10s 20ms/step - loss: 0.5935 - acc: 0.7118 - val_loss: 0.5135 - val_acc: 0.7891
Epoch 7/100
510/510 [==============================] - 10s 20ms/step - loss: 0.5572 - acc: 0.7471 - val_loss: 0.4739 - val_acc: 0.7969
Epoch 8/1

In [50]:
predictions = model.predict(xvalid_pad)
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.88
Average recall score: 0.94


In [51]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(2))
model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='acc', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, callbacks=[earlystop])

Epoch 1/100
510/510 [==============================] - 9s 18ms/step - loss: 0.7245 - acc: 0.4941
Epoch 2/100
510/510 [==============================] - 8s 16ms/step - loss: 0.6889 - acc: 0.5471
Epoch 3/100
510/510 [==============================] - 8s 16ms/step - loss: 0.6787 - acc: 0.5725
Epoch 4/100
510/510 [==============================] - 8s 16ms/step - loss: 0.6470 - acc: 0.6039
Epoch 5/100
510/510 [==============================] - 8s 16ms/step - loss: 0.6264 - acc: 0.6490
Epoch 6/100
510/510 [==============================] - 8s 16ms/step - loss: 0.5900 - acc: 0.6765
Epoch 7/100
510/510 [==============================] - 8s 16ms/step - loss: 0.5462 - acc: 0.7294
Epoch 8/100
510/510 [==============================] - 8s 16ms/step - loss: 0.5113 - acc: 0.7490
Epoch 9/100
510/510 [==============================] - 8s 16ms/step - loss: 0.4965 - acc: 0.7922
Epoch 10/100
510/510 [==============================] - 8s 16ms/step - loss: 0.4798 - acc: 0.7765
Epoch 11/100
510/510 [=======

In [52]:
predictions = model.predict(xvalid_pad)
from sklearn.metrics import average_precision_score, recall_score
average_precision = average_precision_score(yvalid, np.argmax(predictions, axis=1))
average_recall_score = recall_score(yvalid, np.argmax(predictions, axis=1))

print('Average precision score: {0:0.2f}'.format(
      average_precision))
print('Average recall score: {0:0.2f}'.format(
      average_recall_score))

Average precision score: 0.90
Average recall score: 0.94
